In [93]:
%reset

In [94]:
# 加载所需的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [96]:
def initialization(parameter):
    #产生随机的工作表
    joblist = pd.DataFrame(columns=['d_min','d_max','w_o','w_f','p','w','start','stop'],
                           dtype = int)
    #Trapezoidal Suitability Function，打分函数，列标题代表区块开始的时间点
    tsf = pd.DataFrame(0,
                       index=range(parameter['time_units']),
                       columns=range(parameter['n']))
    # 依次生存每一个任务的属性
    for i in range(parameter['n']):
        #时间段：w,d_min,d_max,d
        d_min = np.random.randint(parameter['d_min'],parameter['d_max']+1)
        d_max = np.random.randint(d_min,parameter['d_max']+1)
        w = np.random.randint(d_max,parameter['w_max']+1)
        #时间点：w_o,w_f,start,stop
        w_o = np.random.randint(0,parameter['time_units']-w)
        w_f = w_o + w
        #优先级：p
        p = np.random.randint(1,parameter['p_max']+1)
        # 更新joblist
        joblist.loc[i] = (d_min,d_max,w_o,w_f,p,w,0,0)

        # 更新打分函数
        s_i = tsf[i].copy()
        # 梯形4个顶点
        a = w_o
        b = int( w_o + np.ceil((w - d_max)/2) )
        c = b + d_max
        d = w_f
        #赋值1
        s_i[range(b,c)] = 1
        #赋值2
        if b > a:
            degree_1 = 1/(2*(b-a))
            n_1 = 1
            for j in range(a,b):
                s_i[j] = degree_1 * (2 * n_1 - 1)
                n_1 += 1
        #赋值3
        if d > c:
            degree_2 = 1/(2*(d-c))
            n_2 = 1
            for j in range(d-1,c-1,-1):
                s_i[j] = degree_2 * (2 * n_2 - 1)
                n_2 += 1
        #更新打分函数
        tsf[i] = s_i

    return tsf,joblist

In [99]:
def pd_sequencing(parameter,pd_jobindex,pd_job):
    #总时间轴上1代表位置可用
    pd_timetable = pd.Series(1,index=range(parameter['time_units']))
    #任务时间轴上1代表在窗口中
    empty_timetable = pd.Series(0,index=range(parameter['time_units']))
    #建立空工作表，按顺序存放被选择的工作的序号
    chosen_job_list = list()
    #对每一个工作进行判断
    for i in range(parameter['n']):
        job_index = pd_jobindex[i]
        job_info = pd_job.loc[job_index,]

        d_min = job_info.loc['d_min']
        d_max = job_info.loc['d_max']
        w_o = job_info.loc['w_o']
        w_f = job_info.loc['w_f']
        p = job_info.loc['p']
        w = job_info.loc['w']
        #下序列用来存放每一个开始时间，长度为d_min的工作带来的收益
        position_value = pd.Series(0,index=range(parameter['time_units']))
        #假设没有可供使用的位置
        at_least_one = False
        #对每一个位置进行测试
        for j in range(w-d_min+1):
            test_start = w_o+j
            test_stop = test_start+d_min
            test_job = empty_timetable.copy()
            test_job.loc[range(test_start,test_stop)]=1
            #判断位置是否可用
            if np.sum(test_job*pd_timetable) == d_min:
                at_least_one = True
                position_value.loc[test_start] =\
                    np.sum(TSF.loc[range(test_start,test_stop),job_index])
        #如果有至少一个可用位置，则选择最大的那个
        if at_least_one:
            decide_start = \
                np.min(position_value.index[position_value == position_value.max()])
            decide_stop = decide_start+d_min
            #更新工作表
            pd_job.loc[job_index,'start'] = decide_start
            pd_job.loc[job_index,'stop'] = decide_stop
            #更新工作序号表
            chosen_job_list.append(job_index)
        #刚刚被占用的位置不再可用
        pd_timetable.loc[range(decide_start,decide_stop)]=0

    return pd_job,chosen_job_list

In [101]:
def pd_timing(pd_chosen_jobindex,pd_job):
    test_pd_job = pd_job.copy()
    #对每一个工作进行左右延伸
    for i in range(len(pd_chosen_jobindex)):
        job_index = pd_chosen_jobindex[i]
        job_info = test_pd_job.loc[job_index,]

        d_min = job_info.loc['d_min']
        d_max = job_info.loc['d_max']
        w_o = job_info.loc['w_o']
        w_f = job_info.loc['w_f']
        p = job_info.loc['p']
        w = job_info.loc['w']
        start = job_info.loc['start']
        stop = job_info.loc['stop']
        #左右边界
        new_start = np.nanmax([test_pd_job.loc[test_pd_job.loc[:,'stop']<=start,'stop'].max(),
                               w_o,stop-d_max])
        new_stop = np.nanmin([test_pd_job.loc[test_pd_job.loc[:,'start']>=stop,'start'].min(),
                              w_f,new_start+d_max])
        #左右延伸
        test_pd_job.loc[job_index,'start'] = new_start
        test_pd_job.loc[job_index,'stop'] = new_stop
    return test_pd_job

In [103]:
def get_timetable(job,parameter):
    empty_timetable = pd.Series(-1,index=range(parameter['time_units']))
    for i in range(parameter['n']):
        empty_timetable.loc[range(job.loc[i,'start'],job.loc[i,'stop'])]=i
    return empty_timetable

In [106]:
def score(JOB,TSF,parameter):
    s = 0
    for i in range(parameter['n']):
        start = JOB.loc[i,'start']
        stop = JOB.loc[i,'stop']
        p = JOB.loc[i,'p']

        s_i = 0
        for j in range(start,stop):
            s_i += TSF.loc[j,i]

        s += s_i * p
    return s

In [95]:
# 设置参数
parameter = {'n':40,
             'time_units' : 100,
             'w_max':25,
             'd_min':1,
             'd_max':25,
             'p_max':10}

In [113]:
#初始化
np.random.seed(1)
TSF,JOB = initialization(parameter)

In [114]:
#PD算法：任务按照p值排序
pd_job = JOB.copy()
pd_jobindex = JOB.sort_values(by='p',ascending = False).index

In [115]:
pd_job,pd_chosen_jobindex = pd_sequencing(parameter,pd_jobindex,pd_job)

In [116]:
pd_job_after_timing = pd_timing(pd_chosen_jobindex,pd_job)

In [117]:
pd_s = score(pd_job,TSF,parameter)